In [1]:
# import tensoflow.keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
# from PIL import Image

In [2]:
np.random.seed(1337)

#-- Cargar sólo las imágenes sin las etiquetas ---
(x_train, _), (x_test, _) = mnist.load_data()
input_dim = 28*28
x_train = np.reshape(x_train, [-1, input_dim])/255.0
x_test = np.reshape(x_test,   [-1, input_dim])/255.0

print(x_train.shape,x_test.shape,input_dim)

(60000, 784) (10000, 784) 784


In [3]:

# # This is the size of our encoded representations
# encoding_dim = 32  # 32 flotantes -> factor de compresion 24.5, asumiendo una entrada de 28x28=784

# ##-------- ENCODER ---------------
# encoder_input = Input(shape=(input_dim,), name='encoder_input')
# code = Dense(128, activation='relu', name='codDense01')(encoder_input)
# code = Dense(64, activation='relu', name='codDense02')(code)
# code = Dense(encoding_dim, name='latent_vector')(code)
# encoder = Model(encoder_input, code, name='encoder')

# ##------- DECODER -----------
# latent_input = Input(shape=(encoding_dim,), name='decoder_input')
# decoded_image = Dense(64, activation='relu')(latent_input)
# decoded_image = Dense(128, activation='relu')(decoded_image)
# decoded_image = Dense(input_dim,activation="sigmoid",name='decoder_output')(decoded_image)
# decoder = Model(latent_input, decoded_image, name='decoder')

# # Este modelo mapea una entrada en su reconstrucción
# autoencoder = Model(encoder_input, decoder(encoder(encoder_input)), name='autoencoder')
# print(autoencoder.summary()) 
# print(encoder.summary()) 

input_img = layers.Input(shape=(784,), name='encoder_input')
encoded = layers.Dense(128, activation='relu')(input_img)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu', name='latent_vector')(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(784, activation='sigmoid',name='decoder_output')(decoded)

autoencoder = Model(input_img, decoded)
print(autoencoder.summary()) 

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
latent_vector (Dense)        (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
decoder_output (Dense)       (None, 784)              

In [4]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Predict the Autoencoder output from test images
# x_decoded = autoencoder.predict(x_test)


Epoch 1/100
235/235 [==============================] - 4s 16ms/step - loss: 0.2472 - val_loss: 0.1683
Epoch 2/100
235/235 [==============================] - 3s 12ms/step - loss: 0.1512 - val_loss: 0.1391
Epoch 3/100
235/235 [==============================] - 4s 15ms/step - loss: 0.1324 - val_loss: 0.1261
Epoch 4/100
235/235 [==============================] - 3s 14ms/step - loss: 0.1226 - val_loss: 0.1177
Epoch 5/100
235/235 [==============================] - 3s 14ms/step - loss: 0.1162 - val_loss: 0.1128
Epoch 6/100
235/235 [==============================] - 3s 15ms/step - loss: 0.1115 - val_loss: 0.1082
Epoch 7/100
235/235 [==============================] - 4s 16ms/step - loss: 0.1082 - val_loss: 0.1060
Epoch 8/100
235/235 [==============================] - 4s 16ms/step - loss: 0.1058 - val_loss: 0.1037
Epoch 9/100
235/235 [==============================] - 4s 15ms/step - loss: 0.1037 - val_loss: 0.1014
Epoch 10/100
235/235 [==============================] - 4s 15ms/step - loss: 0.101

235/235 [==============================] - 3s 15ms/step - loss: 0.0801 - val_loss: 0.0800
Epoch 81/100
235/235 [==============================] - 4s 15ms/step - loss: 0.0801 - val_loss: 0.0799
Epoch 82/100
235/235 [==============================] - 3s 14ms/step - loss: 0.0800 - val_loss: 0.0799
Epoch 83/100
235/235 [==============================] - 4s 15ms/step - loss: 0.0799 - val_loss: 0.0797
Epoch 84/100
235/235 [==============================] - 3s 14ms/step - loss: 0.0799 - val_loss: 0.0796
Epoch 85/100
235/235 [==============================] - 3s 15ms/step - loss: 0.0799 - val_loss: 0.0796
Epoch 86/100
235/235 [==============================] - 4s 15ms/step - loss: 0.0798 - val_loss: 0.0795
Epoch 87/100
235/235 [==============================] - 3s 15ms/step - loss: 0.0798 - val_loss: 0.0796
Epoch 88/100
235/235 [==============================] - 3s 14ms/step - loss: 0.0798 - val_loss: 0.0794
Epoch 89/100
235/235 [==============================] - 3s 14ms/step - loss: 0.0797 - 

In [6]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoded.predict(x_test)
decoded_imgs = decoded.predict(encoded_imgs)

n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()




AttributeError: 'Tensor' object has no attribute 'predict'